In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Korean MRC Baseline

## Dependency
다음과 같은 라이브러리를 사용한다.
- [Konlpy](https://konlpy.org/ko/latest/index.html): 파이썬 한국어 NLP 처리기
- [Mecab-korean](https://bitbucket.org/eunjeon/mecab-ko-dic/src): 한국어 형태소 분석기

In [ ]:
! apt-get install -y openjdk-8-jdk python3-dev
! pip install konlpy "tweepy<4.0.0"
! /bin/bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
openjdk-8-jdk is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 94 not upgraded.
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


## 데이터셋 구성
현재 JSON 데이터를 볼 수 있는 클래스를 하나 작성하자.

In [ ]:
from typing import List, Tuple, Dict, Any
import json
import random

class KoMRC:
    def __init__(self, data, indices: List[Tuple[int, int, int]]):
        self._data = data
        self._indices = indices

    # Json을 불러오는 메소드
    @classmethod
    def load(cls, file_path: str):
        with open(file_path, 'r', encoding='utf-8') as fd:
            data = json.load(fd)

        indices = []
        for d_id, document in enumerate(data['data']):
            for p_id, paragraph in enumerate(document['paragraphs']):
                for q_id, _ in enumerate(paragraph['qas']):
                    indices.append((d_id, p_id, q_id))
        
        return cls(data, indices)

    # 데이터 셋을 잘라내는 메소드
    @classmethod
    def split(cls, dataset, eval_ratio: float=.1, seed=42):
        indices = list(dataset._indices)
        random.seed(seed)
        random.shuffle(indices)
        train_indices = indices[int(len(indices) * eval_ratio):]
        eval_indices = indices[:int(len(indices) * eval_ratio)]

        return cls(dataset._data, train_indices), cls(dataset._data, eval_indices)

    def __getitem__(self, index: int) -> Dict[str, Any]:
        d_id, p_id, q_id = self._indices[index]
        paragraph = self._data['data'][d_id]['paragraphs'][p_id]

        context = paragraph['context']
        qa = paragraph['qas'][q_id]

        guid = qa['guid']
        question = qa['question']
        answers = qa['answers']

        return {
            'guid': guid,
            'context': context,
            'question': question,
            'answers': answers
        }

    def __len__(self) -> int:
        return len(self._indices)


`load` 메소드를 이용해서 Json 데이터를 불러올 수 있다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


PATH =  '/content/drive/MyDrive/Goorm_Deep_Learning/Projects/project2'

dataset = KoMRC.load(PATH+'/Data/train.json')
print("Number of Samples:", len(dataset))
print(dataset[0])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of Samples: 12037
{'guid': '798db07f0b9046759deed9d4a35ce31e', 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.', 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?', 'answer

In [ ]:
#빠른 데이터 실행을 위해 데이터 자르기
use_dataset, not_use_dataset = KoMRC.split(dataset,0.8)
print("Number of Use Samples:", len(use_dataset))
print("Number of not Use Samples:", len(not_use_dataset))
print(use_dataset[0])

Number of Use Samples: 2408
Number of not Use Samples: 9629
{'guid': 'e8a80bfc972b46589c28846b8c12b395', 'context': '디즈니 애니메이션 ‘겨울왕국’이 개봉 17일 만에 국내 역대 애니메이션 흥행 기록을 갈아치웠다. 2일 영화진흥위원회 영화관입장권통합전산망에 따르면 ‘겨울왕국’은 전날까지 544만명의 관객을 모아 ‘쿵푸팬더 2’(506만명)를 제치고 역대 애니메이션 흥행 신기록을 세웠다. 흥행 속도도 빨랐다. ‘겨울왕국’은 개봉 4일 만에 100만 관객을 돌파했고 9일 만에 200만, 11일 만에 300만 관객을 넘었다. 영화가 히트하면서 오리지널사운드트랙(OST) 앨범 ‘프로즌’(FROZEN)도 덩달아 인기다. 타이틀곡 ‘렛 잇 고’(Let it Go)는 벅스뮤직 일간 음원차트, 멜론의 실시간 차트에서 1위를 차지하는 등 각종 국내 음원차트에서 상위권에 올랐다. 이 영화를 수입·배급하는 소니 픽처스 릴리징 코리아는 ‘겨울왕국’이 조만간 뮤지컬 영화 최고 흥행 기록을 보유한 ‘레미제라블’(591만명)마저 넘어설 것으로 보고 있다. ‘겨울왕국’은 동화나라 왕국에 사는 마법을 가진 언니와 밝고 용감한 동생 간의 자매애를 다뤘다.', 'question': "겨울왕국' 개봉 이전까지 누적 관객수가 가장 많았던 애니메이션은?", 'answers': [{'text': '‘쿵푸팬더 2’', 'answer_start': 109}, {'text': '쿵푸팬더 2', 'answer_start': 110}]}


`split` 메소드를 이용하면 데이터 셋을 나눌 수 있다.

In [ ]:
train_dataset, dev_dataset = KoMRC.split(use_dataset)
print("Number of Train Samples:", len(train_dataset))
print("Number of Dev Samples:", len(dev_dataset))
print(dev_dataset[0])

Number of Train Samples: 2168
Number of Dev Samples: 240
{'guid': 'f83b43309b9749cbb025aa6a72c7c882', 'context': '이 8.3 인민소비품 생산 규정으로 인하여 1989년부터 현재인 2020년까지 한번도 중단되지 않고 가내작업반이 유지될 수 있도록 허가를 하였고 특히 사적 기업 설립권을 법제화를 하여 인정하는 것을 확정하는 계기가 되었다.\n\n하지만 결국 제대로된 법안이 설립이 되지 못하여 조선민주주의인민공화국의 내각의 규정으로 존재만 하여 결국 상업법의 개정이 절실히 중요하였고 이때 사회주의 상업법을 개정을 하였으며 이때 2004년에 사회주의 상업법에 가내작업반에 대한 법안이 수록되었다.\n\n중앙상업지도기관과 지방정권기관은 주민지역의 특성과 주민의 수요에 맞게 전문, 종합서비스망을 합리적으로 배치하며 서비스 업종을 실정에 맞게 정해주어야 한다고 되어 있으며 상점, 식당, 서비스업소를 운영하려는 해당 기관과 기업소 및 협동 단체는 해당 상업지도기관의 영업허가를 받아야 하며 영업활동을 정해진 질서대로 하여야 한다고 수록이 되어 있다.\n\n지방정권기관과 편의봉사기관과 기업소 및 협동 단체는 편의서비스 가내작업반과 가내편의서비스직원을 널리 운영하여야 한다고 되어 있으며 편의서비스 가내작업반과 가내편의서비스직원의 조직과 배치는 상업지도기관이 하게 되어 있다. \n\n국가계획기관과 무역기관, 해당 기관은 편의서비스용 자재와 부속품을 계획에 맞물려 보장하고 상업지도기관은 편의서비스용 자재, 부속품을 정상적으로 공급하여야 한다고 되어 있어 현재 가내작업반이 서로 무역을 하며 자재를 공급받을 수 있도록 법적으로 허용이 되어 있다.', 'question': '가내작업반에 대한 법률의 안건이 사회주의 상업법에 기록된 해는?', 'answers': [{'text': '2004년', 'answer_start': 229}, {'text': '2004', 'answer_start': 229}]}

단어 단위로 토큰화해서 정답 위치를 찾기 위하여 토큰화 및 위치 인덱싱을 하는 클래스를 상속을 통해 작성해 보자.

In [ ]:
from typing import Generator

import konlpy

class TokenizedKoMRC(KoMRC):
    def __init__(self, data, indices: List[Tuple[int, int, int]]) -> None:
        super().__init__(data, indices)
        self._tagger = konlpy.tag.Mecab()

    def _tokenize_with_position(self, sentence: str) -> List[Tuple[str, Tuple[int, int]]]:
        position = 0
        tokens = []
        for morph in self._tagger.morphs(sentence):
            position = sentence.find(morph, position)
            tokens.append((morph, (position, position + len(morph))))
            position += len(morph)
        return tokens
            
    def __getitem__(self, index: int) -> Dict[str, Any]:
        sample = super().__getitem__(index)

        context, position = zip(*self._tokenize_with_position(sample['context']))
        context, position = list(context), list(position)
        question = self._tagger.morphs(sample['question'])

        if sample['answers'] is not None:
            answers = []
            for answer in sample['answers']:
                for start, (position_start, position_end) in enumerate(position):
                    if position_start <= answer['answer_start'] < position_end:
                        break
                else:
                    print(context, answer)
                    raise ValueError("No mathced start position")

                target = ''.join(answer['text'].split(' '))
                source = ''
                for end, morph in enumerate(context[start:], start):
                    source += morph
                    if target in source:
                        break
                else:
                    print(context, answer)
                    raise ValueError("No Matched end position")

                answers.append({
                    'start': start,
                    'end': end
                })
        else:
            answers = None
        
        return {
            'guid': sample['guid'],
            'context_original': sample['context'],
            'context_position': position,
            'question_original': sample['question'],
            'context': context,
            'question': question,
            'answers': answers
        }
        

In [ ]:
PATH =  '/content/drive/MyDrive/Goorm_Deep_Learning/Projects/project2'

dataset = KoMRC.load(PATH+'/Data/train.json')

train_dataset, dev_dataset = TokenizedKoMRC.split(use_dataset)
print("Number of Train Samples:", len(train_dataset))
print("Number of Dev Samples:", len(dev_dataset))
print(dev_dataset[0])

Number of Train Samples: 2168
Number of Dev Samples: 240
{'guid': 'f83b43309b9749cbb025aa6a72c7c882', 'context_original': '이 8.3 인민소비품 생산 규정으로 인하여 1989년부터 현재인 2020년까지 한번도 중단되지 않고 가내작업반이 유지될 수 있도록 허가를 하였고 특히 사적 기업 설립권을 법제화를 하여 인정하는 것을 확정하는 계기가 되었다.\n\n하지만 결국 제대로된 법안이 설립이 되지 못하여 조선민주주의인민공화국의 내각의 규정으로 존재만 하여 결국 상업법의 개정이 절실히 중요하였고 이때 사회주의 상업법을 개정을 하였으며 이때 2004년에 사회주의 상업법에 가내작업반에 대한 법안이 수록되었다.\n\n중앙상업지도기관과 지방정권기관은 주민지역의 특성과 주민의 수요에 맞게 전문, 종합서비스망을 합리적으로 배치하며 서비스 업종을 실정에 맞게 정해주어야 한다고 되어 있으며 상점, 식당, 서비스업소를 운영하려는 해당 기관과 기업소 및 협동 단체는 해당 상업지도기관의 영업허가를 받아야 하며 영업활동을 정해진 질서대로 하여야 한다고 수록이 되어 있다.\n\n지방정권기관과 편의봉사기관과 기업소 및 협동 단체는 편의서비스 가내작업반과 가내편의서비스직원을 널리 운영하여야 한다고 되어 있으며 편의서비스 가내작업반과 가내편의서비스직원의 조직과 배치는 상업지도기관이 하게 되어 있다. \n\n국가계획기관과 무역기관, 해당 기관은 편의서비스용 자재와 부속품을 계획에 맞물려 보장하고 상업지도기관은 편의서비스용 자재, 부속품을 정상적으로 공급하여야 한다고 되어 있어 현재 가내작업반이 서로 무역을 하며 자재를 공급받을 수 있도록 법적으로 허용이 되어 있다.', 'context_position': [(0, 1), (2, 3), (3, 4), (4, 5), (6, 8), (8, 11), (12, 14), (15, 17), (17, 19), (20, 22), (22, 23), (24, 28), (28

In [ ]:
sample = dev_dataset[0]
print(sample['context'][sample['answers'][0]['start']:sample['answers'][0]['end']+1])

['2004', '년']


## Vocab 생성 및 Indexing
토큰화된 데이터 셋을 기준으로 Vocab을 만들고 인덱싱을 하는 `Indexer`를 만들자.

In [ ]:
from typing import Sequence
from collections import Counter
from itertools import chain

from tqdm.notebook import tqdm

class Indexer:
    def __init__(self,
        id2token: List[str], 
        max_length: int=1024,
        pad: str='<pad>', unk: str='<unk>', cls: str='<cls>', sep: str='<sep>'
    ):
        self.pad = pad
        self.unk = unk
        self.cls = cls
        self.sep = sep
        self.special_tokens = [pad, unk, cls, sep]

        self.max_length = max_length

        self.id2token = self.special_tokens + id2token
        self.token2id = {token: token_id for token_id, token in enumerate(self.id2token)}

    @property
    def vocab_size(self):
        return len(self.id2token)
    
    @property
    def pad_id(self):
        return self.token2id[self.pad]
    @property
    def unk_id(self):
        return self.token2id[self.unk]
    @property
    def cls_id(self):
        return self.token2id[self.cls]
    @property
    def sep_id(self):
        return self.token2id[self.sep]

    @classmethod
    def build_vocab(cls,
        dataset: TokenizedKoMRC, 
        min_freq: int=5
    ):
        counter = Counter(chain.from_iterable(
            sample['context'] + sample['question']
            for sample in tqdm(dataset, desc="Counting Vocab")
        ))

        return cls([word for word, count in counter.items() if count >= min_freq])
    
    def decode(self,
        token_ids: Sequence[int]
    ):
        return [self.id2token[token_id] for token_id in token_ids]

    def sample2ids(self,
        sample: Dict[str, Any],
    ) -> Dict[str, Any]:
        context = [self.token2id.get(token, self.unk_id) for token in sample['context']]
        question = [self.token2id.get(token, self.unk_id) for token in sample['question']]

        context = context[:self.max_length-len(question)-3]             # Truncate context
        
        input_ids = [self.cls_id] + question + [self.sep_id] + context + [self.sep_id]
        token_type_ids = [0] * (len(question) + 1) + [1] * (len(context) + 2)

        if sample['answers'] is not None:
            answer = sample['answers'][0]
            start = min(answer['start'] + len(question) + 2, self.max_length - 1)
            end = min(answer['end'] + len(question) + 2, self.max_length - 1)
        else:
            start = None
            end = None

        return {
            'guid': sample['guid'],
            'context': sample['context_original'],
            'question': sample['question_original'],
            'position': sample['context_position'],
            'input_ids': input_ids,
            'token_type_ids': token_type_ids,
            'start': start,
            'end': end
        }

In [ ]:
indexer = Indexer.build_vocab(dataset)
print(indexer.sample2ids(dev_dataset[0]))

Counting Vocab:   0%|          | 0/12037 [00:00<?, ?it/s]

{'guid': 'f83b43309b9749cbb025aa6a72c7c882', 'context': '이 8.3 인민소비품 생산 규정으로 인하여 1989년부터 현재인 2020년까지 한번도 중단되지 않고 가내작업반이 유지될 수 있도록 허가를 하였고 특히 사적 기업 설립권을 법제화를 하여 인정하는 것을 확정하는 계기가 되었다.\n\n하지만 결국 제대로된 법안이 설립이 되지 못하여 조선민주주의인민공화국의 내각의 규정으로 존재만 하여 결국 상업법의 개정이 절실히 중요하였고 이때 사회주의 상업법을 개정을 하였으며 이때 2004년에 사회주의 상업법에 가내작업반에 대한 법안이 수록되었다.\n\n중앙상업지도기관과 지방정권기관은 주민지역의 특성과 주민의 수요에 맞게 전문, 종합서비스망을 합리적으로 배치하며 서비스 업종을 실정에 맞게 정해주어야 한다고 되어 있으며 상점, 식당, 서비스업소를 운영하려는 해당 기관과 기업소 및 협동 단체는 해당 상업지도기관의 영업허가를 받아야 하며 영업활동을 정해진 질서대로 하여야 한다고 수록이 되어 있다.\n\n지방정권기관과 편의봉사기관과 기업소 및 협동 단체는 편의서비스 가내작업반과 가내편의서비스직원을 널리 운영하여야 한다고 되어 있으며 편의서비스 가내작업반과 가내편의서비스직원의 조직과 배치는 상업지도기관이 하게 되어 있다. \n\n국가계획기관과 무역기관, 해당 기관은 편의서비스용 자재와 부속품을 계획에 맞물려 보장하고 상업지도기관은 편의서비스용 자재, 부속품을 정상적으로 공급하여야 한다고 되어 있어 현재 가내작업반이 서로 무역을 하며 자재를 공급받을 수 있도록 법적으로 허용이 되어 있다.', 'question': '가내작업반에 대한 법률의 안건이 사회주의 상업법에 기록된 해는?', 'position': [(0, 1), (2, 3), (3, 4), (4, 5), (6, 8), (8, 11), (12, 14), (15, 17), (17, 19), (20, 22), (22, 23), (24, 28), (28, 29), (29, 31), (32, 3

쉽게 Indexer를 활용하기 위해 Indexer가 포함된 데이터 셋을 만들자.

In [ ]:
class IndexerWrappedDataset:
    def __init__(self, dataset: TokenizedKoMRC, indexer: Indexer) -> None:
        self._dataset = dataset
        self._indexer = indexer

    def __len__(self) -> int:
        return len(self._dataset)
    
    def __getitem__(self, index: int) -> Dict[str, Any]:
        sample = self._indexer.sample2ids(self._dataset[index])
        sample['attention_mask'] = [1] * len(sample['input_ids'])

        return sample


In [ ]:
indexed_train_dataset = IndexerWrappedDataset(train_dataset, indexer)
indexed_dev_dataset = IndexerWrappedDataset(dev_dataset, indexer)

sample = indexed_dev_dataset[0]
print(sample['input_ids'], sample['attention_mask'], sample['token_type_ids'], sample['start'], sample['end'])

[2, 1, 1, 17, 1, 1, 59, 1, 39, 1, 1, 17, 1, 214, 89, 57, 175, 3, 39, 113, 23, 86, 1, 1, 1, 1, 1, 1, 5, 1, 32, 1, 1, 120, 1, 32, 1, 95, 234, 16, 1, 109, 28, 168, 92, 1, 1, 39, 1, 43, 152, 56, 1, 1, 111, 79, 362, 92, 1, 34, 155, 1, 1, 525, 117, 1, 188, 111, 79, 5, 1, 79, 57, 119, 117, 1, 79, 57, 1, 10, 109, 150, 22, 23, 1, 1, 1, 214, 1, 39, 1, 39, 109, 28, 1, 5, 1, 1, 1, 59, 1, 59, 1, 1, 1, 106, 79, 5, 1, 1, 59, 1, 39, 1, 1, 79, 362, 92, 1, 1, 1, 117, 1, 117, 79, 1, 1, 1, 32, 17, 1, 1, 17, 1, 1, 17, 1, 1, 39, 1, 109, 150, 22, 23, 1, 1, 16, 1, 100, 1, 1, 1, 30, 1, 1, 59, 1, 100, 1, 59, 1, 17, 536, 134, 1, 87, 1, 1, 45, 117, 1, 155, 1, 1, 79, 139, 1, 1, 117, 1, 17, 536, 134, 1, 15, 1, 1, 109, 127, 56, 1, 1, 87, 1, 87, 1, 1, 111, 1, 79, 1, 1, 1, 100, 1, 201, 67, 1, 1, 57, 1, 1, 16, 1, 59, 1, 1, 111, 447, 1, 79, 139, 1, 1, 117, 1, 1, 1, 79, 1, 1, 1, 39, 109, 127, 56, 22, 23, 1, 1, 1, 100, 344, 59, 1, 1, 100, 1, 201, 67, 1, 1, 57, 1, 1, 1, 1, 100, 1, 1, 1, 1, 117, 1, 1, 79, 1, 1, 109, 127, 56

## Transformer Encoder를 활용한 MRC 모델
![Bert for MRC](https://miro.medium.com/max/340/1*cXDOP0gsE7Zp8-sgZqYfTA.png)

Transformer 인코더 마지막에 Linear Layer를 붙여 정답의 시작과 끝을 맞추는 간단한 모델을 생성보자.

In [ ]:
!pip install transformers
!pip install wandb

In [ ]:
#wandb 로그인 하기. 링크(https://wandb.ai/authorize) 에 들어가서  API key를 복사한 다음 붙여넣기
!wandb login
#682d4bece1af96de531dc3acdf980f281a2328a1

wandb: Currently logged in as: obok (use `wandb login --relogin` to force relogin)


## Sweep 사용하기 1 : Define the Sweep

In [ ]:
import wandb

sweep_config = {
  "name" : "my-sweep6",
  "method" : "grid",
  "parameters" : {
    "learning_rate" :{
       "values" : [1e-3, 1e-4, 1e-5]
    },
    "batch_size" :{
        "values" : [128,64,32]
    }
  }
}

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric
sweep_id = wandb.sweep(sweep_config)


Create sweep with ID: 21om6mu1
Sweep URL: https://wandb.ai/obok/uncategorized/sweeps/21om6mu1


In [ ]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'name': 'my-sweep6',
 'parameters': {'batch_size': {'values': [128, 64, 32]},
                'learning_rate': {'values': [0.001, 0.0001, 1e-05]}}}


## wandb.init : Call wandb.init() at the top of script to start a new run
새 작업을 초기화하기 위해 스크립트 시작 부분에서 한번 호출. 
W&B에서 new run이 생성되고 wandb.ai에 데이터를 동기화하기 위한 백그라운드 프로세스가 시작됨-> 
라이브 시각화를 볼 수 있음

Arguments : 
*  project - new  run을 보내는 프로젝트 이름, 프로젝트를 지정할 경우 web에서 프로젝트 생성 후 init 실행 
*  entity - new run을 보내는 사용자 이름 or 팀 이름 
* resume - 재개 동작 설정 옵션 (allow, must, never,auto,none)
* id -  재개에 사용되는 이 실행의 고유 ID, 프로젝트에서 고유해야하며 실행을 삭제하면 ID 를 재사용할 수 없음 


=> 자세한 설명 :https://docs.wandb.ai/ref/python/init 

In [ ]:
#wandb.init(project="Machine Reading Comprehension", entity="obok",resume = "allow",id="experiment_02")


In [ ]:
import torch.nn as nn

from transformers.models.bert.modeling_bert import (
    BertModel,
    BertPreTrainedModel
)

## Simple Version for Bert QA: https://huggingface.co/transformers/_modules/transformers/models/bert/modeling_bert.html#BertForQuestionAnswering.forward
class BertForQuestionAnswering(BertPreTrainedModel):
    _keys_to_ignore_on_load_unexpected = [r"pooler"]

    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config, add_pooling_layer=False)
        self.start_linear = nn.Linear(config.hidden_size, 1)
        self.end_linear = nn.Linear(config.hidden_size, 1)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        start_logits = self.start_linear(outputs.last_hidden_state).squeeze(-1)
        end_logits = self.end_linear(outputs.last_hidden_state).squeeze(-1)

        return start_logits, end_logits

## 학습 준비

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

class Collator:
    def __init__(self, indexer: Indexer) -> None:
        self._indexer = indexer

    def __call__(self, samples: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        samples = {
            key: [sample[key] for sample in samples]
            for key in samples[0]
        }

        for key in 'start', 'end':
            if samples[key][0] is None:
                samples[key] = None
            else:
                samples[key] = torch.tensor(samples[key], dtype=torch.long)
        for key in 'input_ids', 'attention_mask', 'token_type_ids':
            samples[key] = pad_sequence(
                [torch.tensor(sample, dtype=torch.long) for sample in samples[key]],
                batch_first=True, padding_value=self._indexer.pad_id
            )

        return samples

In [ ]:
import torch
from transformers import BertConfig

In [ ]:
import os
from statistics import mean

import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_

import matplotlib.pyplot as plt

import time


##wandb.log : Keep track of metrics, videos, custom plots, and more
metrics, media, or custom objects을 step 마다 기록하기 위해 호출
wandb.log(dict)이 기록할 때 마다 default로 step이 증가하므로 시간이 지남에 따라 모델과 데이터가 어떻게 발전하는지 확인 가능

https://docs.wandb.ai/guides/track/log


In [ ]:
accumulation = 4
def training( Model, train_loader, dev_loader, epochs, optimizer):
    torch.manual_seed(42)
    train_losses = []
    dev_losses = []
    model = Model
    optimizer = optimizer
    start_train_epoch = 1

    model.cuda()
    step = 0

    for epoch in range(start_train_epoch, epochs+1):
        start_time = time.time()
        print("Epoch", epoch)
        # Training
        running_loss = 0.
        losses = []
        progress_bar = tqdm(train_loader, desc='Train')
        for batch in progress_bar:
            del batch['guid'], batch['context'], batch['question'], batch['position']
            batch = {key: value.cuda() for key, value in batch.items()}
            start = batch.pop('start')
            end = batch.pop('end')
    
            start_logits, end_logits = model(**batch)
            loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)
            (loss / accumulation).backward()
            running_loss += loss.item()
            del batch, start, end, start_logits, end_logits, loss
    
            step += 1
            if step % accumulation:
                continue

            clip_grad_norm_(model.parameters(), max_norm=1.)
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

            losses.append(running_loss / accumulation)
            running_loss = 0.
            progress_bar.set_description(f"Train - Loss: {losses[-1]:.3f}")

        train_losses.append(mean(losses))
        print(f"train score: {train_losses[-1]:.3f}")
        training_time = (time.time() - start_time)
        print("Training Time per epoch: {:.4f}sec".format(training_time))

        #wandb.log({"train_loss": self.train_losses[-1]})
        
        # Evaluation
        losses = []
        for batch in tqdm(dev_loader, desc="Evaluation"):
            del batch['guid'], batch['context'], batch['question'], batch['position']
            batch = {key: value.cuda() for key, value in batch.items()}
            start = batch.pop('start')
            end = batch.pop('end')
    
            with torch.no_grad():
                start_logits, end_logits = model(**batch)
            loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)

            losses.append(loss.item())
            del batch, start, end, start_logits, end_logits, loss
        dev_losses.append(mean(losses))
        print(f"Evaluation score: {dev_losses[-1]:.3f}")

        #model.save_pretrained(f'{output_path}/model.{epoch}')
        val_time = (time.time() - training_time - start_time)
        print("Validation Time per epoch: {:.4f}sec".format(val_time))

        wandb.log({"train_loss": train_losses[-1],
                    "val_loss": dev_losses[-1]})
        wandb.watch(model)
        print("All Task Time per epoch: {:.4f}sec".format(time.time() - start_time))
    
    avg_loss = sum(dev_losses)/epochs
    

    return avg_loss

def draw_loss(self):
    t = list(range(1, 31))
    plt.plot(t, self.train_losses, label="Train Loss")
    plt.plot(t, self.dev_losses, label="Dev Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

##wandb.config : Use a dictionary-like object to save experiment configuration

hyperparameters, dataset name, model type 등과 같은 입력 설정, 실험을 위한 기타 독립 변수 등 학습에 사용한 구성을 저장하도록 스크립트에서 개체를 설정.

실험을 분석하고 향후 작업을 재현하는데 유용함

https://docs.wandb.ai/guides/track/config


In [ ]:
import gc
gc.collect()

50

In [ ]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def build_dataset(batch_size):
  accumulation = 4 # 메모리를 아끼기 위하여 Gradient accumulation을 해보자
  
  collator = Collator(indexer)
  train_loader = DataLoader(indexed_train_dataset, batch_size=batch_size//accumulation, shuffle=True, collate_fn=collator, num_workers=2)
  dev_loader = DataLoader(indexed_dev_dataset, batch_size=batch_size//accumulation, shuffle=False, collate_fn=collator, num_workers=2)

  return train_loader,dev_loader

def build_optimizer(Model, learning_rate):
  optimizer = torch.optim.AdamW(Model.parameters(), lr=learning_rate)
  return optimizer


def build_Model():
    config = BertConfig(
        vocab_size=indexer.vocab_size,
        max_position_embeddings=1024,
        hidden_size=256,
        num_hidden_layers=4,
        num_attention_heads=4,
        intermediate_size=1024
    )
    model = BertForQuestionAnswering(config)

    return model.to(device)


def train(config=None):
    output_path = PATH+'/dump'
    epochs = 3
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        train_loader, dev_loader =  build_dataset(config.batch_size)
        Model = build_Model()
        optimizer = build_optimizer(Model,config.learning_rate)

        avg_loss = training(Model,train_loader, dev_loader, epochs, optimizer)
        print(avg_loss)
        wandb.log({"loss": avg_loss})
        gc.collect() #안쓰는 메모리 정리
        torch.cuda.empty_cache() ## cuda delete cache


In [ ]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: h6eatrg4 with config:
wandb: 	batch_size: 128
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: obok (use `wandb login --relogin` to force relogin)


Epoch 1


Train:   0%|          | 0/68 [00:00<?, ?it/s]

train score: 11.518
Training Time per epoch: 20.1677sec


Evaluation:   0%|          | 0/8 [00:00<?, ?it/s]

...

Train:   0%|          | 0/136 [00:00<?, ?it/s]

train score: 10.556
Training Time per epoch: 19.9610sec


Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

Evaluation score: 11.208
Validation Time per epoch: 1.1096sec
All Task Time per epoch: 21.0725sec
11.173101827833387



loss,▁
train_loss,█▃▁
val_loss,█▁▆
loss,11.1731
train_loss,10.55618
val_loss,11.20775


In [ ]:
trainer.draw_loss()

NameError: ignored

학습 데이터 셋에 Overfitting이 일어나는 것을 확인할 수 있다.

## Answer Inference
모델의 Output을 활용해서 질문의 답을 찾는 코드를 작성하자.

In [ ]:
model = BertForQuestionAnswering.from_pretrained('dump/model.30')
model.cuda()
model.eval()

In [ ]:
for idx, sample in zip(range(1, 4), indexed_train_dataset):
    print(f'------{idx}------')
    print('Context:', sample['context'])
    print('Question:', sample['question'])
    
    input_ids, token_type_ids = [
        torch.tensor(sample[key], dtype=torch.long, device="cuda")
        for key in ("input_ids", "token_type_ids")
    ]
    
    with torch.no_grad():
        start_logits, end_logits = model(input_ids=input_ids[None, :], token_type_ids=token_type_ids[None, :])
    start_logits.squeeze_(0), end_logits.squeeze_(0)
    
    start_prob = start_logits[token_type_ids.bool()][1:-1].softmax(-1)
    end_prob = end_logits[token_type_ids.bool()][1:-1].softmax(-1)
    probability = torch.triu(start_prob[:, None] @ end_prob[None, :])
    index = torch.argmax(probability).item()
    
    start = index // len(end_prob)
    end = index % len(end_prob)
    
    start = sample['position'][start][0]
    end = sample['position'][end][1]

    print('Answer:', sample['context'][start:end])

## Test 출력 파일 작성

In [ ]:
test_dataset = TokenizedKoMRC.load('/kaggle/input/k-digital-goorm-3-korean-mrc/test.json')
test_dataset = IndexerWrappedDataset(test_dataset, indexer)
print("Number of Test Samples", len(test_dataset))
print(test_dataset[0])

In [ ]:
import csv

os.makedirs('out', exist_ok=True)
with torch.no_grad(), open('out/baseline.csv', 'w') as fd:
    writer = csv.writer(fd)
    writer.writerow(['Id', 'Predicted'])

    rows = []
    for sample in tqdm(test_dataset, "Testing"):
        input_ids, token_type_ids = [
            torch.tensor(sample[key], dtype=torch.long, device="cuda")
            for key in ("input_ids", "token_type_ids")
        ]
    
        with torch.no_grad():
            start_logits, end_logits = model(input_ids=input_ids[None, :], token_type_ids=token_type_ids[None, :])
        start_logits.squeeze_(0), end_logits.squeeze_(0)
    
        start_prob = start_logits[token_type_ids.bool()][1:-1].softmax(-1)
        end_prob = end_logits[token_type_ids.bool()][1:-1].softmax(-1)
        probability = torch.triu(start_prob[:, None] @ end_prob[None, :])
        index = torch.argmax(probability).item()
    
        start = index // len(end_prob)
        end = index % len(end_prob)
    
        start = sample['position'][start][0]
        end = sample['position'][end][1]

        rows.append([sample["guid"], sample['context'][start:end]])
    
    writer.writerows(rows)